In [1]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [ ]:
#@title Day 1: Secret Entrance
import math

inputs = fread('input.txt').splitlines()

def rotate(pos, d, rotation, part1, part2):
  flip = lambda p : [p%100,(100-p)%100][d=='R']
  pos = flip(pos) + rotation
  return (flip(pos), part1 + (flip(pos) == 0), part2 + math.floor(pos / 100))

acc = (50, 0, 0)
[acc := rotate(acc[0], i[0],int(i[1:]), acc[1], acc[2]) for i in inputs]

print(acc[1])
print(acc[2])

997
5978


In [ ]:
#@title Day 2: Gift Shop
import math, re

inputs = fread('input 2.txt')
inputs = [list(map(int,s.split('-'))) for s in inputs.split(',')]

def bads(lower, upper, pattern):
  some = [i for i in range(lower, upper+1) if re.search(pattern, str(i))]
  return sum([s for s in some if s >= lower and s <= upper])

print(sum([bads(*i,r'^([0-9]+)\1$') for i in inputs]))
print(sum([bads(*i,r'^([0-9]+)\1+$') for i in inputs]))

5398419778
15704845910


In [ ]:
#@title Day 3: Lobby
from itertools import accumulate
import math

inputs = fread('input 3.txt')
inputs = [list(map(int,s)) for s in inputs.splitlines()]

def joltage(line, d):
  def best_idx(prev,j):
    offset = [0,prev+1][prev > -1]
    values = line[offset:len(line)-(d-1-j)]
    return max(range(len(values)), key=values.__getitem__)+offset

  idxs = list(accumulate(range(-1,d), best_idx))[1:]
  return sum([int(math.pow(10,d-1-j)) * line[idxs[j]] for j in range(d)])

print(sum(map(lambda l: joltage(l, 2), inputs)))
print(sum(map(lambda l: joltage(l, 12), inputs)))

17207
170997883706617


In [13]:
#@title Day 4
from itertools import accumulate
import math, re

inputs = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@."""
inputs = fread('input 4.txt')
inputs = inputs.splitlines()

def is_p(prev, x,y):
  if x < 0 or y < 0: return False
  if x >= len(prev[0]): return False
  if y >= len(prev): return False
  return prev[y][x] == '@'

def replace_papers(prev):
  next = []
  count = 0

  for y in range(len(prev)):
    next.append([])

    for x in range(len(prev[0])):
      if not is_p(prev,x,y):
        next[y].append(prev[y][x])
        continue

      ct = is_p(prev, x-1,y-1) + is_p(prev, x,y-1) + is_p(prev, x+1,y-1) + \
            is_p(prev, x-1,y) + is_p(prev, x+1,y) + \
            is_p(prev, x-1,y+1) + is_p(prev, x,y+1) + is_p(prev, x+1,y+1)

      next[y].append('x' if ct < 4 else '@')
      count += ct < 4

  return (count, next)

cont = True
curr = inputs
total = 0
while cont:
  (removed, curr) = replace_papers(curr)
  total += removed
  cont = removed != 0

print(total)




9401
